In [ ]:
!pip install selenium webdriver-manager
!pip install --upgrade pandas lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 98.4 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all 

In [ ]:
# impor pustaka
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import by
from selenium.webdriver.support.ui import webdriverwait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import elementclickinterceptedexception, timeoutexception, webdriverexception
import time
import os

# url target
url_the = 'https://www.timeshighereducation.com/world-university-rankings/2025/subject-ranking/computer-science#:~:text=Discover%20the%20best%20Computer%20Science%20schools%20in%20the,ranks%201%2C122%20institutions%20from%2083%20countries%20and%20territories.'

# opsi untuk menjalankan browser di latar belakang (headless)
options = webdriver.chromeoptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920,1080")

# inisialisasi driver
driver = webdriver.chrome(options=options)

try:
    driver.get(url_the)
    wait = webdriverwait(driver, 60)

    # coba hapus dialog cookie dengan javascript
    try:
        driver.execute_script("var element = document.getelementbyid('cybotcookiebotdialog'); if (element) { element.parentnode.removechild(element); }")
        time.sleep(2)
    except webdriverexception:
        pass # lanjutkan jika gagal

    # tunggu hingga tabel peringkat muncul
    table_id = "datatable-1"
    wait.until(ec.presence_of_element_located((by.id, table_id)))

    # klik dropdown untuk menampilkan semua data
    try:
        select_element_xpath = f"//select[@name='{table_id}_length']"
        select_element = wait.until(ec.element_to_be_clickable((by.xpath, select_element_xpath)))
        select_element.click()

        all_option_xpath = f"{select_element_xpath}/option[text()='all']"
        all_option = wait.until(ec.element_to_be_clickable((by.xpath, all_option_xpath)))
        all_option.click()

        # tunggu hingga jumlah baris data bertambah (menandakan semua data sudah dimuat)
        wait.until(ec.presence_of_element_located((by.xpath, f"//table[@id='{table_id}']/tbody/tr[26]")))

        # ambil html dan baca tabel dengan pandas
        html_source = driver.page_source
        df_the = pd.read_html(html_source, attrs={'id': table_id})[0]

        # simpan ke csv dan tampilkan pratinjau
        df_the.to_csv('the_2025_scraped_final.csv', index=False)
        print("pratinjau data the yang berhasil diambil:")
        display(df_the.head())
        print("\ndata berhasil disimpan ke 'the_2025_scraped_final.csv'")

    except (timeoutexception, elementclickinterceptedexception) as e:
        print(f"error: gagal berinteraksi dengan halaman: {e}")
    except exception as e:
        print(f"terjadi error tak terduga: {e}")

finally:
    driver.quit()

Membuka halaman https://www.timeshighereducation.com/world-university-rankings/2025/subject-ranking/computer-science#:~:text=Discover%20the%20best%20Computer%20Science%20schools%20in%20the,ranks%201%2C122%20institutions%20from%2083%20countries%20and%20territories....
Attempting to remove cookie dialog using JavaScript...
JavaScript executed to remove cookie dialog.
Waiting for table with ID 'datatable-1' to appear...
✅ Ranking table detected.
Attempting to click the dropdown to show all data...
Clicked the dropdown.
Changing view to display all data...
✅ All data successfully loaded.


/tmp/ipython-input-2-3637962662.py:72: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_the = pd.read_html(html_source, attrs={'id': table_id})[0]



Successfully retrieved data from THE page!


,Rank,Name Country/Region,Overall,Research Quality,Industry,International Outlook,Research Environment,Teaching
0,1,University of OxfordUnited Kingdom,98.3,99.4,92.4,95.5,98.7,99.2
1,2,University of CambridgeUnited Kingdom,97.3,95.6,99.5,96.1,100.0,95.8
2,3,Massachusetts Institute of TechnologyUnited St...,96.2,99.1,100.0,80.6,99.4,93.2
3,4,ETH ZurichSwitzerland,96.1,99.3,90.8,94.4,99.4,91.3
4,5,Stanford UniversityUnited States,96.0,99.8,100.0,76.7,98.8,93.5



📊 Data successfully saved to 'the_2025_scraped_final.csv'

Browser closed.


In [ ]:
# url target
url_cwur = 'https://cwur.org/2025.php'

# konfigurasi chrome untuk mode headless
chrome_options = webdriver.chromeoptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# inisialisasi webdriver
driver = webdriver.chrome(options=chrome_options)

try:
    driver.get(url_cwur)
    # beri waktu agar halaman dimuat sepenuhnya
    time.sleep(5)

    # ambil html dan baca tabel
    html_source = driver.page_source
    list_of_tables = pd.read_html(html_source)

    if list_of_tables:
        df_cwur = list_of_tables[0]

        # simpan ke csv dan tampilkan pratinjau
        df_cwur.to_csv('cwur_2025_scraped_selenium.csv', index=False)
        print("pratinjau data cwur yang berhasil diambil:")
        print(df_cwur.head())
        print(f"\ndata berhasil disimpan ke 'cwur_2025_scraped_selenium.csv'")
    else:
        print("gagal menemukan tabel di halaman.")

finally:
    driver.quit()